In [27]:
import time
import string
import numpy as np
import itertools
import os, random
import pandas as pd
import matplotlib.pylab as plt
import requests
from bs4 import BeautifulSoup as bs

class Team(object):
    num_teams = 0
    name_teams = []
    def __init__(self, name):
        self.name = name
        self.players = players[self.name]
        self.abs_points = abs_points[self.name]
        self.position = 0
        self.league_points = 0
        self.goals_scored = 0
        self.goals_taken = 0
        self.vic_draw_losses = {}
        
        Team.num_teams += 1
        Team.name_teams.append(self.name)
        
    def get_name(self):
        return self.name
        
    def get_players_list(self):
        return self.players
        
    def get_abs_points(self):
        return self.abs_points
        
    def get_position(self):
        return self.position
        
    def get_goals_scored(self):
        return self.goals_scored
    
    def set_goals_scored(self, goals):
        self.goals_scored += goals

    def get_goals_taken(self):
        return self.goals_scored
    
    def set_goals_taken(self, goals):
        self.goals_taken += goals
    
    def get_num_of_teams():
        return Team.num_teams
        
    def get_all_teams():
        return Team.name_teams
    
    def get_league_points(self):
        return self.league_points
    
    def set_league_points(self,points):
        self.league_points += points
        

        
    def points_update(self,day):
        #Update points of team up to day
        self.league_points = 0
        for i in range(1,day+1):
            if self.vic_draw_losses[i] == 'V':
                self.league_points += 3
            elif self.vic_draw_losses[i] == 'L':
                self.league_points += 0
            elif self.vic_draw_losses[i] == 'D':
                self.league_points += 1
                
                
    def victory(self,day):
        self.vic_draw_losses[day] = 'V'
        self.points_update(day)
        
    def loss(self,day):
        self.vic_draw_losses[day] = 'L'
        self.points_update(day)
        
    def draw(self,day):
        self.vic_draw_losses[day] = 'D'
        self.points_update(day)
        
        
class Match(object):
    
    def __init__(self, team1, team2, day):
        self.team1 = team1
        self.team2 = team2
        self.day = day
                
    def play():
        
        points1 = self.team1.abs_points
        points2 = self.team2.abs_points

        temp1 = int(((points1[day - 1] - 66) // 6) + 1)
        temp2 = int(((points2[day - 1] - 66) // 6) + 1)

        if temp1 < 0:
            temp1 = 0
        if temp2 < 0:
            temp2 = 0
        
        self.team1.set_goals_scored(temp1)
        self.team2.set_goals_scored(temp2)
        
        if temp1 > temp2:
            self.team1.victory(self.day)
            self.team2.loss(self.day)
        elif temp1 < temp2:
            self.team2.victory(self.day)
            self.team1.loss(self.day) 
        else:
            self.team2.draw(self.day)
            self.team1.draw(self.day)
            
class Day(object):
    # class defines a Day, consisting of several Matches objects
    
    def __init__(self, list_matches):
        self.list_matches = list_matches
        
    def play(self):
        for i in self.list_matches:
            i.play()

class League(object):
    
    def __init__(self, calendar):
        
        #il calendario é generato come una lista di tuple, ogni giornata é una tuple contenente n_teams/2 partite
        self.calendar = calendar
        self.days = []
        
        for i in self.calendar:
            m = []
            for j in i:
                m.append(Match(j[0],j[1]))
                
            d = Day(m)
            
            self.days.append(d)
            
    def play(self):
        
        for i in self.days:
            i.play()






In [2]:
import fanta_calendario as fc

In [3]:
teams, players, abs_points = fc.scraping('fantascandalo')

In [4]:
teams

['FC ROXY',
 'FC Pastaboy',
 'Bucalina FC',
 'Fc Stress',
 'Ciolle United',
 'AC PICCHIA',
 'FC BOMBAGALLO',
 'LA CORRAZZATA POTEMKIN']

In [12]:
from fanta_calendario import *
import random

def create_league_random(teams,n_leagues):
    
    n_teams = len(teams)
    L = []
    L_all = []
    matches, days = create_matches_days(teams)

    while len(L_all) < n_leagues:

        days0 = days.copy()
        L0 = []

        for i in range(n_teams-1):

            if len(days0):
                random_day = days0[random.randint(1,len(days0))-1]
                L0.append(random_day)
                days0 = get_compatible_lists2(random_day, days0, n_teams)

        if len(L0) == n_teams - 1:
            L_all.append(L0)


In [13]:
test = create_league_random(teams,1)


[('FC ROXY', 'Bucalina FC')]


In [15]:
range(len(teams)//2)

range(0, 4)

In [56]:
n_teams = len(teams)
L = []
L_all = []
matches, days = create_matches_days(teams)

while len(L_all) < n_leagues:

    days0 = days.copy()
    L0 = []

    for i in range(n_teams-1):

        if len(days0):
            random_day = days0[random.randint(1,len(days0))-1]
            L0.append(random_day)
            days0 = get_compatible_lists2(random_day, days0, n_teams)

    if len(L0) == n_teams - 1:
        L_all.append(L0)


In [45]:

c = 0
for i in L_all:
    c += 1
    print('Campionato {}\n'.format(c))
    for j in i:
        for k in j:
            print(k)
#             print('\n')
        print('\n')

Campionato 1

('FC ROXY', 'Bucalina FC')
('FC Pastaboy', 'FC BOMBAGALLO')
('Fc Stress', 'LA CORRAZZATA POTEMKIN')
('Ciolle United', 'AC PICCHIA')


('FC ROXY', 'AC PICCHIA')
('FC Pastaboy', 'Fc Stress')
('Bucalina FC', 'FC BOMBAGALLO')
('Ciolle United', 'LA CORRAZZATA POTEMKIN')


('FC ROXY', 'Ciolle United')
('FC Pastaboy', 'AC PICCHIA')
('Bucalina FC', 'Fc Stress')
('FC BOMBAGALLO', 'LA CORRAZZATA POTEMKIN')


('FC ROXY', 'AC PICCHIA')
('FC Pastaboy', 'LA CORRAZZATA POTEMKIN')
('Bucalina FC', 'Ciolle United')
('Fc Stress', 'FC BOMBAGALLO')


('FC ROXY', 'LA CORRAZZATA POTEMKIN')
('FC Pastaboy', 'Bucalina FC')
('Fc Stress', 'AC PICCHIA')
('Ciolle United', 'FC BOMBAGALLO')


('FC ROXY', 'FC Pastaboy')
('Bucalina FC', 'FC BOMBAGALLO')
('Fc Stress', 'LA CORRAZZATA POTEMKIN')
('Ciolle United', 'AC PICCHIA')


('FC ROXY', 'Fc Stress')
('FC Pastaboy', 'AC PICCHIA')
('Bucalina FC', 'LA CORRAZZATA POTEMKIN')
('Ciolle United', 'FC BOMBAGALLO')


Campionato 2

('FC ROXY', 'FC Pastaboy')
('Bucal

In [57]:
len(L_all)

778

In [48]:
def print_days(days):

    for i in days:
        for j in i:
            print(j)
    #         print('\n')
        print('\n')  

In [58]:
print_days(L_all)

(('FC ROXY', 'Ciolle United'), ('FC Pastaboy', 'FC BOMBAGALLO'), ('Bucalina FC', 'LA CORRAZZATA POTEMKIN'), ('Fc Stress', 'AC PICCHIA'))
(('FC ROXY', 'AC PICCHIA'), ('FC Pastaboy', 'Bucalina FC'), ('Fc Stress', 'LA CORRAZZATA POTEMKIN'), ('Ciolle United', 'FC BOMBAGALLO'))
(('FC ROXY', 'Fc Stress'), ('FC Pastaboy', 'Ciolle United'), ('Bucalina FC', 'FC BOMBAGALLO'), ('AC PICCHIA', 'LA CORRAZZATA POTEMKIN'))
(('FC ROXY', 'FC Pastaboy'), ('Bucalina FC', 'Fc Stress'), ('Ciolle United', 'LA CORRAZZATA POTEMKIN'), ('AC PICCHIA', 'FC BOMBAGALLO'))
(('FC ROXY', 'LA CORRAZZATA POTEMKIN'), ('FC Pastaboy', 'AC PICCHIA'), ('Bucalina FC', 'Ciolle United'), ('Fc Stress', 'FC BOMBAGALLO'))
(('FC ROXY', 'FC BOMBAGALLO'), ('FC Pastaboy', 'LA CORRAZZATA POTEMKIN'), ('Bucalina FC', 'AC PICCHIA'), ('Fc Stress', 'Ciolle United'))
(('FC ROXY', 'Bucalina FC'), ('FC Pastaboy', 'Fc Stress'), ('Ciolle United', 'AC PICCHIA'), ('FC BOMBAGALLO', 'LA CORRAZZATA POTEMKIN'))


(('FC ROXY', 'AC PICCHIA'), ('FC Pastab